In [ ]:
import sqlite3
from datetime import datetime

DB_FILE = "ntpu_paper.sqlite"

def get_conn():
    conn = sqlite3.connect(DB_FILE)
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

# --- 最小必要：商品主表（欄位先留基本，之後再加都可） ---
def create_table_test(conn):
    conn.execute("""
    CREATE TABLE IF NOT EXISTS test (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item_id TEXT UNIQUE,   -- 你的產品ID；之後所有關聯都用它
        url TEXT,
        title TEXT,
        description TEXT,
        crawl_time DATETIME
        -- 想到再加：hashtags TEXT, price_text TEXT, sold_text TEXT, shop_name TEXT, raw_json TEXT ...
    );
    """)
    conn.execute("CREATE INDEX IF NOT EXISTS idx_tests_item_id ON test(item_id);")


def upsert_test(conn, item_id, url=None, title=None, description=None, crawl_time=None):
    crawl_time = crawl_time or datetime.now().isoformat()
    conn.execute("""
    INSERT INTO test (item_id, url, title, description, crawl_time)
    VALUES (?, ?, ?, ?, ?)
    ON CONFLICT(item_id) DO UPDATE SET
        url=excluded.url,
        title=excluded.title,
        description=excluded.description,
        crawl_time=excluded.crawl_time;
    """, (item_id, url, title, description, crawl_time))
    # 若已建立 FTS5，順手同步
    try:
        conn.execute("""
        INSERT INTO test_fts(rowid, title, description)
        SELECT id, title, description FROM test WHERE item_id=?
        ON CONFLICT(rowid) DO UPDATE SET
            title=excluded.title,
            description=excluded.description;
        """, (item_id,))
    except sqlite3.OperationalError:
        # 還沒建 FTS5 就略過
        pass

if __name__ == "__main__":
    conn = get_conn()
    # 1) 先建商品主表（最小）
    create_table_test(conn)
    upsert_test(conn,
                item_id="demo_001",
                url="https://shopee.tw/demo",
                title="可口可樂 500ml",
                description="冰涼好喝，可口可樂經典款。")
    conn.commit()
    conn.close()
    print(f"OK -> 建好 {DB_FILE} （含 test 表；demo_001 已寫入）。")

OK -> 建好 ntpu_paper.sqlite （含 test 表；demo_001 已寫入）。
